<a href="https://colab.research.google.com/github/SukhmanpreetKaurManes/THAPAR-KAGGLE-3/blob/main/competition3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# === Load Datasets ===
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
sample_submission = pd.read_csv('sample_submission.csv', delimiter=',', on_bad_lines='skip')

# === Drop unnecessary columns ===
train = train.drop(columns=['Row#'])
test = test.drop(columns=['Row#'])

# === Prepare Features and Target ===
X = train.drop(columns=['output'])
y = train['output']
X_test = test.drop(columns=['id'])

# === Scale Features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# === K-Fold Cross-Validation ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    score = r2_score(y_val, preds)
    r2_scores.append(score)
    print(f"Fold {fold+1} R² Score: {score:.4f}")

print("✅ Average R² Score across folds:", np.mean(r2_scores))

# === Train Final Model on All Data ===
final_model = RandomForestRegressor(n_estimators=100, random_state=42)
final_model.fit(X_scaled, y)

# === Predict on Test Data ===
final_preds = final_model.predict(X_test_scaled)

# === Save Final Submission File ===
sample_submission['output'] = final_preds
sample_submission.to_csv('sukh.csv', index=False)
print("✅ sukh.csv created!")

Fold 1 R² Score: 0.9232
Fold 2 R² Score: 0.9122
Fold 3 R² Score: 0.9113
Fold 4 R² Score: 0.9297
Fold 5 R² Score: 0.9148
✅ Average R² Score across folds: 0.918237640224041
✅ sukh.csv created!


In [ ]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# === Load your data ===
train = pd.read_csv("t.csv").drop(columns=['Row#'])
test = pd.read_csv("te.csv").drop(columns=['Row#', 'id'])
sample_submission = pd.read_csv("sample_submission.csv")

X = train.drop(columns=['output'])
y = train['output']
test_ids = pd.read_csv("te.csv")['id']

# === Scale Features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(test)

# === Baseline: DummyRegressor to force positive R² ===
dummy = DummyRegressor(strategy='mean')
dummy.fit(X_scaled, y)
dummy_preds = dummy.predict(X_scaled)
baseline_r2 = r2_score(y, dummy_preds)
print("Baseline Dummy R² Score:", baseline_r2)

# === Grid Search with RandomForestRegressor ===
param_grid = {
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid = GridSearchCV(
    RandomForestRegressor(n_estimators=100, random_state=42),
    param_grid,
    cv=5,
    scoring='r2'
)
grid.fit(X_scaled, y)

print("Best GridSearchCV R² Score:", grid.best_score_)
print("Best Params:", grid.best_params_)

# === Predict and Save Submission File ===
final_preds = grid.predict(X_test_scaled)
sample_submission['output'] = final_preds.astype(int)
sample_submission.to_csv("gridsearch_submission.csv", index=False)
print("✅ Submission file 'gridsearch_submission.csv' created.")


FileNotFoundError: [Errno 2] No such file or directory: 't.csv'

In [ ]:


import numpy as np
!pip install xgboost

import xgboost as xgb

# === Prepare Features and Target (already done in the preceding code) ===
# X_scaled, y, X_test_scaled are available

# === XGBoost K-Fold Cross-Validation ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores_xgb = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # Use XGBoost Regressor
    model_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                                n_estimators=100,
                                learning_rate=0.1,
                                max_depth=5,
                                random_state=42)

    model_xgb.fit(X_train, y_train)
    preds_xgb = model_xgb.predict(X_val)
    score_xgb = r2_score(y_val, preds_xgb)
    r2_scores_xgb.append(score_xgb)
    print(f"Fold {fold+1} XGBoost R² Score: {score_xgb:.4f}")

print("✅ Average XGBoost R² Score across folds:", np.mean(r2_scores_xgb))


# === Train Final XGBoost Model on All Data ===
final_model_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                                   n_estimators=100,
                                   learning_rate=0.1,
                                   max_depth=5,
                                   random_state=42)
final_model_xgb.fit(X_scaled, y)

# === Predict on Test Data with XGBoost ===
final_preds_xgb = final_model_xgb.predict(X_test_scaled)

# Ensure predictions are integers if the output is expected to be integer-like
final_preds_xgb = final_preds_xgb.round().astype(int)


# === Save Final Submission File (using XGBoost predictions) ===
# sample_submission is already loaded from preceding code
sample_submission['output'] = final_preds_xgb
sample_submission.to_csv('xgb_submission.csv', index=False)
print("✅ xgb_submission.csv created!")


# === Optional: XGBoost Grid Search (replace RandomForest Grid Search) ===
# If you want to optimize XGBoost hyperparameters
# param_grid_xgb = {
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'n_estimators': [100, 200, 300],
#     'subsample': [0.7, 1.0],
#     'colsample_bytree': [0.7, 1.0]
# }

# grid_xgb = GridSearchCV(
#     xgb.XGBRegressor(objective='reg:squareəerror', random_state=42),
#     param_grid_xgb,
#     cv=5,
#     scoring='r2',
#     n_jobs=-1 # Use all available cores
# )
# grid_xgb.fit(X_scaled, y)

# print("Best GridSearchCV XGBoost R² Score:", grid_xgb.best_score_)
# print("Best Params XGBoost:", grid_xgb.best_params_)

# === Predict and Save Submission File with XGBoost Grid Search (if used) ===
# final_preds_grid_xgb = grid_xgb.predict(X_test_scaled)
# sample_submission['output'] = final_preds_grid_xgb.round().astype(int) # Adjust if output is not integer
# sample_submission.to_csv("xgb_gridsearch_submission.csv", index=False)
# print("✅ Submission file 'xgb_gridsearch_submission.csv' created.")
```

SyntaxError: invalid syntax (ipython-input-13-2144675461.py, line 84)

In [ ]:
# Read the file as plain text to inspect its content
try:
    with open('sample_submission.csv', 'r') as f:
        file_content = f.read()
    print(file_content)
except FileNotFoundError:
    print("Error: sample_submission.csv not found.")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")

id,output
1,5974
2,7378
3,7092
4,8038
5,8824
6,6575
7,8364
8,6663
9,5211
10,6242
11,6479
12,5973
13,6369
14,7252
15,7619
16,7607
17,8593
18,8650
19,8715
20,6121
21,6248
22,4208
23,5515
24,8678
25,7232
26,6567
27,6813
28,6158
29,7268
30,8514
31,8859
32,5925
33,7642
34,5167
35,4419
36,5038
37,6760
38,4607
39,7195
40,7294
41,6082
42,5096
43,6265
44,8379
45,6720
46,6952
47,7065
48,6793
49,7150
50,4465
51,4668
52,7273
53,4857
54,5685
55,7761
56,7718
57,5528
58,8037
59,7025
60,4589
61,4597
62,5148
63,5610
64,6376
65,4711
66,6220
67,6900
68,7795
69,4778
70,4343
71,5561
72,8863
73,7453
74,8849
75,5831
76,5145
77,4683
78,8773
79,5784
80,8594
81,7390
82,8113
83,6345
84,6078
85,5514
86,4238
87,5323
88,5381
89,4133
90,4412
91,5445
92,6177
93,8277
94,5602
95,6456
96,6236
97,6640
98,5846
99,7124
100,6763
101,6611
102,8562
103,5143
104,6825
105,7145
106,5338
107,5566
108,7044
109,7322
110,5747
111,8956
112,6981
113,7395
114,7539
115,8692
116,8653
117,4155
118,8187
119,4605
120,8089
121,4771
122,5170


In [ ]:


import numpy as np
!pip install catboost
from catboost import CatBoostRegressor

# === CatBoost K-Fold Cross-Validation ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores_catboost = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # Use CatBoost Regressor
    model_catboost = CatBoostRegressor(iterations=100,
                                       learning_rate=0.1,
                                       depth=5,
                                       random_state=42,
                                       verbose=0) # Set verbose to 0 to suppress output during training

    model_catboost.fit(X_train, y_train)
    preds_catboost = model_catboost.predict(X_val)
    score_catboost = r2_score(y_val, preds_catboost)
    r2_scores_catboost.append(score_catboost)
    print(f"Fold {fold+1} CatBoost R² Score: {score_catboost:.4f}")

print("✅ Average CatBoost R² Score across folds:", np.mean(r2_scores_catboost))

# === Train Final CatBoost Model on All Data ===
final_model_catboost = CatBoostRegressor(iterations=100,
                                         learning_rate=0.1,
                                         depth=5,
                                         random_state=42,
                                         verbose=0)
final_model_catboost.fit(X_scaled, y)

# === Predict on Test Data with CatBoost ===
final_preds_catboost = final_model_catboost.predict(X_test_scaled)

# Ensure predictions are integers if the output is expected to be integer-like
final_preds_catboost = final_preds_catboost.round().astype(int)

# === Save Final Submission File (using CatBoost predictions) ===
# sample_submission is already loaded from preceding code
sample_submission['output'] = final_preds_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)
print("✅ catboost_submission.csv created!")

# === Optional: CatBoost Grid Search ===
# If you want to optimize CatBoost hyperparameters
# param_grid_catboost = {
#     'depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'iterations': [100, 200, 300],
#     'l2_leaf_reg': [1, 3, 5]
# }

# grid_catboost = GridSearchCV(
#     CatBoostRegressor(random_state=42, verbose=0),
#     param_grid_catboost,
#     cv=5,
#     scoring='r2',
#     n_jobs=-1 # Use all available cores
# )
# grid_catboost.fit(X_scaled, y)

# print("Best GridSearchCV CatBoost R² Score:", grid_catboost.best_score_)
# print("Best Params CatBoost:", grid_catboost.best_params_)

# === Predict and Save Submission File with CatBoost Grid Search (if used) ===
# final_preds_grid_catboost = grid_catboost.predict(X_test_scaled)
# sample_submission['output'] = final_preds_grid_catboost.round().astype(int) # Adjust if output is not integer
# sample_submission.to_csv("catboost_gridsearch_submission.csv", index=False)
# print("✅ Submission file 'catboost_gridsearch_submission.csv' created.")

Fold 1 CatBoost R² Score: 0.9273
Fold 2 CatBoost R² Score: 0.9123
Fold 3 CatBoost R² Score: 0.9166
Fold 4 CatBoost R² Score: 0.9335
Fold 5 CatBoost R² Score: 0.9184
✅ Average CatBoost R² Score across folds: 0.9216245675826581
✅ catboost_submission.csv created!


In [ ]:
!pip install xgboost

In [ ]:
!pip install autogluon
from autogluon.tabular import TabularPredictor

# Assuming train and test DataFrames are already loaded and processed
# You might need to adjust column names if they are different

# Define the target variable
label = 'output'

# Initialize TabularPredictor
# 'best_quality' setting trains a variety of models and ensembles them
predictor = TabularPredictor(label=label, eval_metric='r2').fit(train, presets='best_quality')

# Make predictions on the test data
predictions = predictor.predict(test)

# Convert predictions to integer if required
predictions = predictions.round().astype(int)

# Save the submission file
# Assuming sample_submission is already loaded and has an 'id' column
submission_autogluon = sample_submission.copy()
submission_autogluon['output'] = predictions
submission_autogluon.to_csv('autogluon_submission.csv', index=False)

print("✅ autogluon_submission.csv created!")

# You can also get leaderboad to see the performance of different models trained by AutoGluon
leaderboard = predictor.leaderboard(silent=True)
leaderboard


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

No path specified. Models will be saved in: "AutogluonModels/ag-20250709_011357"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.88 GB / 12.67 GB (85.8%)
Disk Space Avail:   65.04 GB / 107.72 GB (60.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be

(_ray_fit pid=7151) [1000]	valid_set's l2: 141486	valid_set's r2: 0.922797
(_ray_fit pid=7307) [1000]	valid_set's l2: 139286	valid_set's r2: 0.919015 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(_dystack pid=6792) 	0.921	 = Validation score   (r2)
(_dystack pid=6792) 	57.85s	 = Training   runtime
(_dystack pid=6792) 	1.39s	 = Validation runtime
(_dystack pid=6792) Fitting model: LightGBM_BAG_L1 ... Training model for up to 528.08s of the 826.24s of remaining time.
(_dystack pid=6792) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.12%)
(_dystack pid=6792) 	0.9234	 = Validation score   (r2)
(_dystack pid=6792) 	44.39s	 = Training   runtime
(_dystack pid=6792) 	0.22s	 = Validation runtime
(_dystack pid=6792) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 478.94s of the 777.11s of remaining time.
(_dystack pid=6792) /usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-lear

✅ autogluon_submission.csv created!


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.925805,r2,8.843771,1841.899952,0.002394,0.504927,3,True,27
1,WeightedEnsemble_L2,0.925383,r2,2.315296,1375.600379,0.001468,0.649298,2,True,16
2,CatBoost_r177_BAG_L2,0.924607,r2,7.281036,1711.187379,0.110571,59.205927,2,True,26
3,CatBoost_BAG_L2,0.924477,r2,7.244207,1720.667668,0.073743,68.686216,2,True,20
4,LightGBM_BAG_L2,0.924329,r2,7.343542,1703.377781,0.173077,51.396329,2,True,18
5,LightGBM_BAG_L1,0.924055,r2,0.147568,43.313611,0.147568,43.313611,1,True,4
6,LightGBMXT_BAG_L2,0.923771,r2,7.574153,1701.804771,0.403688,49.823320,2,True,17
7,CatBoost_BAG_L1,0.923742,r2,0.078168,47.162726,0.078168,47.162726,1,True,6
8,CatBoost_r177_BAG_L1,0.923625,r2,0.075666,42.804476,0.075666,42.804476,1,True,12
9,NeuralNetFastAI_BAG_L2,0.923398,r2,7.580609,1788.825384,0.410145,136.843932,2,True,22


In [ ]:


!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import optuna

# === Load Datasets ===
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# === Drop unnecessary columns ===
train = train.drop(columns=['Row#'])
test = test.drop(columns=['Row#'])

# === Prepare Features and Target ===
X = train.drop(columns=['output'])
y = train['output']
X_test = test.drop(columns=['id'])

# === Scale Features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# === Optuna + RandomForest with K-Fold Cross-Validation (R² Score) ===
def run_optuna_random_forest_kfold():
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'max_depth': trial.suggest_int('max_depth', 5, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
        }

        kf = KFold(n_splits=8, shuffle=True, random_state=42)
        r2_scores = []

        for train_idx, val_idx in kf.split(X_scaled):
            X_train_fold, X_val_fold = X_scaled[train_idx], X_scaled[val_idx]
            y_train_fold, y_val_fold = y[train_idx], y[val_idx]

            model = RandomForestRegressor(random_state=42, **params)
            model.fit(X_train_fold, y_train_fold)
            preds = model.predict(X_val_fold)
            r2 = r2_score(y_val_fold, preds)
            r2_scores.append(r2)

        return -np.mean(r2_scores)  # Minimize the negative average R²

    # Run Optuna Study
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=150)

    print("✅ Best Parameters Found by Optuna:", study.best_params)
    print("✅ Best Mean Validation R² Score (CV):", -study.best_value)

    # Train final model on full training data
    best_model = RandomForestRegressor(random_state=42, **study.best_params)
    best_model.fit(X_scaled, y)

    # 🔍 R² on full training data
    train_preds = best_model.predict(X_scaled)
    final_r2 = r2_score(y, train_preds)
    print("🎯 Final R² Score on Full Training Data:", final_r2)

    return best_model

# === Run the model optimization and training ===
model = run_optuna_random_forest_kfold()

# === Predict on Test Data ===
final_preds = model.predict(X_test_scaled)

# === Save Final Submission File ===
sample_submission['output'] = final_preds
sample_submission.to_csv('lakhbir_1187_kfold.csv', index=False)
print("✅ lakhbir_1187_kfold.csv created!")

[I 2025-07-08 14:07:35,700] A new study created in memory with name: no-name-78177a16-95ae-406b-be5d-29f5467dbe52
[I 2025-07-08 14:10:10,519] Trial 0 finished with value: -0.9223472378638351 and parameters: {'n_estimators': 423, 'max_depth': 22, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 0 with value: -0.9223472378638351.
[I 2025-07-08 14:13:18,545] Trial 1 finished with value: -0.9219058257159869 and parameters: {'n_estimators': 477, 'max_depth': 25, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: -0.9223472378638351.
[I 2025-07-08 14:15:35,226] Trial 2 finished with value: -0.9210912288717408 and parameters: {'n_estimators': 387, 'max_depth': 11, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 0 with value: -0.9223472378638351.
[I 2025-07-08 14:15:58,724] Trial 3 finished with value: -0.9236391474208624 and parameters: {'n_estimators': 118, 'max_depth': 7, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 3 with

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-14-2666848267.py", line 72, in <cell line: 0>
    model = run_optuna_random_forest_kfold()
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-14-2666848267.py", line 55, in run_optuna_random_forest_kfold
    study.optimize(objective, n_trials=150)
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/study.py", line 489, in optimize
    _optimize(
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 64, in _optimize
    _optimize_sequential(
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 161, in _optimize_sequential
    frozen_trial = _run_trial(study, func, catch)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_opti